In [1]:
import pickle
import sys,os
sys.path.append('../OneClickTraining/')
from all_model_py import *
cd = pickle.load(open("../../savedModel/CutDebt/CutDebt.pickle", "rb"))

In [ ]:
cd.re

In [11]:
import jieba
import numpy as np
import sys,os
tpattern_path = '../TimePattern/'
others_pth = '../Others/'
#sys.path.append(tpattern_path)
sys.path.append(tpattern_path)
sys.path.append(others_pth)
from  time_pattern import TimePattern
from others_py import *


class CutDebt:
    
    def __init__(self, **model):
        """
        suggested parameters:
        svc, logistic, nb, jieba_path,tfidf
        """
        self._load_model(**model)
        self.re_time = TimePattern(pattern_path=tpattern_path+'mapping.csv')
        
    def _load_model(self,**model):
        self.svc = model.get('svc')
        self.logistic = model.get('logistic')
        self.nb = model.get('nb')
        self.tfidf = model.get('tfidf')
        self.other = model.get('other')
        self.other.classify('')
        # load jieba
        jieba_path = model.get('jieba_path')
        if jieba_path is not None:
            jieba.load_userdict(jieba_path)
        
        
    def process_time(self, sentence,lower_bounder=36, upper_bounder=24*15):
        time_extract = self.re_time.process(sentence)
        time_label = 0
        if len(time_extract) == 0:
            time_label = 0
        elif len(time_extract) > 1:
            time_label = 1
        else:
            delta = time_extract[0]['gapH']
            if delta < lower_bounder:
                time_label = 2
            elif lower_bounder <= delta < upper_bounder:
                time_label = 3
            else:
                time_label = 4
        return time_label
        
    def classify(self, sentence,lower_bounder=36, upper_bounder=24*15):
        sentence = jieba.cut(sentence, cut_all = False)
        sentence = ' '.join(sentence)
        matrix = self.tfidf.transform([sentence])
        
        result = np.vstack((self.svc.predict_proba(matrix),
                                 self.logistic.predict_proba(matrix),
                                 self.nb.predict_proba(matrix)))
        
        av_pred = np.mean(result, axis = 0)
        max_pred = np.max(av_pred, axis = 0)
        max_arg = np.argmax(av_pred)
        threshold = 0.5
        if np.max(max_pred)<threshold:
            label = 2
        else:
            label = max_arg
        if label == 2:
            response = self.other.classify(sentence)
            label = response['label']
        
        dictionary = {'label': label, 'pred_prob': result, 'av_pred': av_pred}
        return dictionary

In [12]:

cd_dev = CutDebt(svc=cd.svc, logistic=cd.logistic, nb=cd.nb, tfidf=cd.tfidf,other=cd.other)

In [19]:
cd_dev.process_time('一会儿就还')

0

In [5]:
pickle.dump(ifk, open("../../savedModel/WillingToPay/WillingToPay.pickle", "wb"))

In [18]:
cd_dev.re_time.process('一会儿就还')

[]